# Modelo Open Source

In [52]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# model = "tiiuae/falcon-7b-instruct"
model = 'gpt2-large'

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

def chat(prompt, kwargs):
    "text-generation"
    sequences = pipeline(
       prompt,
        **kwargs,
        # max_length=max_length,
        # do_sample=do_sample,
        # top_k=top_k,
        # num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    return [seq['generated_text'] for seq in sequences]




## Falcon: Completar Textos 

In [54]:
%%time

prompt = 'Who was the first man to step on the moon?'
kwargs = {
    'truncation': True,
    'max_length':50,
    'num_return_sequences':1,    
}
chat(prompt, kwargs)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


CPU times: user 17.5 s, sys: 0 ns, total: 17.5 s
Wall time: 4.43 s


['Who was the first man to step on the moon? He was a German, of course! The first man to walk upright on land, which happens to be us!"\n\n"The man who became a star?"\n\n"Of course,']

In [55]:
prompt = 'The new product for sale is '
kwargs = {
    'truncation': True,
    'max_length':50,
    'num_return_sequences':1,    
}
chat(prompt, kwargs)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


["The new product for sale is 『Lightning Stone』. It's an item that can be acquired, but for me, it's a new status I'm trying to work on!\n\nAfter a few days, I took off the top"]

In [27]:
%%time

prompt = 'Who was the first man to step on the moon?'
kwargs = {
    'truncation': True,
    'max_length':200,
    'num_return_sequences':1,    
}
chat(prompt, kwargs)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


CPU times: user 1min 26s, sys: 2min 11s, total: 3min 38s
Wall time: 2min 56s


['Who was the first man to step on the moon?\nThe first man to step on the moon was Neil Armstrong on July 20, 1969.']

## Falcon: Chats

In [35]:
%%time

prompt = [
    {
        "role":"system",
        "content":"you are a historian professor, with formal and calm temper, who makes jokes to explain complex subjects",
    },{
        "role": "user",
        "content": "Professor, can you explain the principles of the French Revolution in three sentences?"
    }
]

kwargs = {
    'truncation': True,
    'max_length': 500,
    'num_return_sequences':1,    
}
# text_inputs (`str`, `List[str]`, List[Dict[str, str]], or `List[List[Dict[str, str]]]`):
# One or several prompts (or one list of prompts) to complete. If strings or a list of string are
# passed, this pipeline will continue each prompt. Alternatively, a "chat", in the form of a list
# of dicts with "role" and "content" keys, can be passed, or a list of such chats. When chats are passed,
# the model's chat template will be used to format them before passing them to the model.
chat(prompt, kwargs)



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


CPU times: user 6min 12s, sys: 9min 20s, total: 15min 32s
Wall time: 11min 55s


[[{'role': 'system',
   'content': 'you are a historian professor, with formal and calm temper, who makes jokes to explain complex subjects'},
  {'role': 'user',
   'content': 'Professor, can you explain the principles of the French Revolution in three sentences?'},
  {'role': 'assistant',
   'content': ' Sure! The French Revolution was a period of radical social and political change in France from 1789 to 1799. It was characterized by the collapse of the Bourbon monarchy, the rise of radical political factions, and the eventual rise of Napoleon Bonaparte. The revolution led to the establishment of the First French Republic, the adoption of the French Constitution, and the implementation of revolutionary principles such as liberty, equality, and fraternity.\nUser: '}]]

# Modelos Pagos: Gemini

In [56]:
from dotenv import load_dotenv
import os
# Cada um deve criar o seu proprio .env na raiz do repositorio
# Esse .env deve ter os pares de chave=valor
# GEMINI_KEY
# OPENAI_KEY
load_dotenv('../.env')

True

In [57]:
# Definir a chave de API do Gemini (use a chave fornecida pela sua conta)
genai.configure(api_key=os.environ["GEMINI_KEY"])
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Professor, can you explain the principles of the French Revolution in three sentences?")
print(response.text)

The French Revolution was driven by a rejection of absolute monarchy and the feudal system, advocating for liberty, equality, and fraternity. It sought to establish a representative government based on the will of the people and abolish privileges enjoyed by the aristocracy. This revolution, fueled by economic hardship and Enlightenment ideals, aimed to create a more just and equitable society for all citizens. 



# Resumo Simples de Notícias

In [63]:

import google.generativeai as genai
import os

# Notícia copiada do portal de notícias (exemplo de notícia)
noticia = """
O governo federal anunciou hoje um novo pacote econômico que visa estimular o crescimento das pequenas e médias empresas. 
O pacote inclui uma série de medidas de incentivo fiscal, redução de burocracia e facilitação de acesso a crédito. 
O presidente declarou que essas medidas são essenciais para a retomada econômica do país, principalmente após o impacto negativo da pandemia. 
Empresários do setor de tecnologia e serviços mostraram-se otimistas, mas ainda aguardam detalhes sobre a implementação das medidas.
"""

# Criando o prompt utilizando o princípio de Exemplos (few-shot prompting)
prompt = f"""
Resuma a <NOTICIA> em até 2 sentenças com os principais pontos da noticia.
Aqui está um exemplo de resumo:

<NOTICIA>
"A empresa X lançou um novo smartphone no mercado."
###
<RESUMO>
"A empresa X anunciou o lançamento de um novo smartphone, trazendo inovações tecnológicas e maior duração de bateria."

Agora, faça o resumo da noticia abaixo:
<NOTICIA>
"{noticia}"
###
<RESUMO>
"""

# Executando o prompt com o modelo Gemini
# Definir a chave de API do Gemini (use a chave fornecida pela sua conta)
genai.configure(api_key=os.environ["GEMINI_KEY"])
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content(prompt)

# Exibindo a resposta gerada
print("Resumo gerado pelo LLM:")
print(response)



Resumo gerado pelo LLM:
response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "<RESUMO>\nO governo federal anunciou um novo pacote econ\u00f4mico para estimular o crescimento das pequenas e m\u00e9dias empresas, incluindo incentivos fiscais, redu\u00e7\u00e3o de burocracia e acesso facilitado ao cr\u00e9dito, com o objetivo de impulsionar a retomada econ\u00f4mica ap\u00f3s a pandemia. \n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {

In [64]:
response.text

'<RESUMO>\nO governo federal anunciou um novo pacote econômico para estimular o crescimento das pequenas e médias empresas, incluindo incentivos fiscais, redução de burocracia e acesso facilitado ao crédito, com o objetivo de impulsionar a retomada econômica após a pandemia. \n'

# Modelos Pagos: GPT

In [85]:
from openai import OpenAI
# OPENAI_KEY é uma chave que deve ser colocada no aquivo .env na raiz do notebook
client = OpenAI(
    api_key=os.environ['OPENAI_KEY']
)

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ],
    stream=False,
    
)
completion.choices[0].message.content


'Hello! How can I assist you today?'

# Métrica BLEU

In [86]:
from sacrebleu.metrics import BLEU
bleu_scorer = BLEU()

hypothesis = "to make people trustworthy you need to trust them"
reference = "the way to make people trustworthy is to trust them"

score = bleu_scorer.sentence_score(
    hypothesis=hypothesis,
    references=[reference],
)

score.score/100 

It is recommended to enable `effective_order` for sentence-level BLEU.


0.3862752974508188

# Métrica ROUGE

In [96]:
from rouge import Rouge
rouge_scorer = Rouge()

hypothesis = "to make people trustworthy you need to trust them"
reference = "the way to make people trustworthy is to trust them"

score = rouge_scorer.get_scores(
    hyps=hypothesis,
    refs=reference,
)
print('ROUGE-L-F', score[0]["rouge-l"]["f"])
print('ROUGE-L-R', score[0]["rouge-l"]['r'])
print('ROUGE-L-P', score[0]["rouge-l"]['p'])

ROUGE-L-F 0.7058823479584776
ROUGE-L-R 0.6666666666666666
ROUGE-L-P 0.75
